In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import cv2
import skimage.exposure
import pysptools.util as util

In [1]:
class Homography_Process:
    
    def __init__(self, folder, number, angle, path_hsi = 'HSI_90', path_tir = 'TIR_90'):
        # folder - номер папки, number - номер изображения, path - директории
        self.folder = folder
        self.number = number
        self.angle = angle
        self.path_hsi = path_hsi
        self.path_tir = path_tir
        
        
    def Plot_RGB_TIR(self, rgb, tir, t_min=0, t_max=0):
        # Отобразить RGB (соответствующего HSI) и TIR (из xlsx таблицы)
        # t_min, t_max - минимум и максимум отображаемой температуры
        
        if t_min == 0 and t_max == 0: t_min, t_max = tir.min(), tir.max()
        
        fig, ax = plt.subplots(1, 2, figsize=(8.5, 5), constrained_layout=True)
        ax[0].axis('off')
        ax[1].axis('off')
        tir_map = LinearSegmentedColormap.from_list("mycmap", ["lawngreen", "yellow", "red"])
        
        ax[0].imshow(rgb)
        print(f'Размер RGB: {rgb.shape}, Минимум: {rgb.min()}, Максимум: {rgb.max()}')
        
        tir_pic = ax[1].imshow(tir, cmap=tir_map, vmin=t_min, vmax=t_max)
        fig.colorbar(tir_pic, ax=ax[1])
        print(f'Размер TIR: {tir.shape}, Минимум: {tir.min()}, Максимум: {tir.max()}')
        
    def Load_RGB_TIR(self, rot_tir_flag=False):
        # Загрузка RGB и TIR
        # rot_tir_flag - если нужно повернуть tir на 180 градусов
        
        self.rgb = cv2.cvtColor(cv2.imread(f'{self.path_hsi}/{self.folder}/{self.number}.png'), cv2.COLOR_BGR2RGB)
        self.rgb = skimage.exposure.rescale_intensity(self.rgb, in_range=(0,50), out_range=(0,255)) # увеличение яркости
            
        
        self.tir = pd.read_excel(f'{self.path_tir}/{self.folder}/{self.number}.xlsx', header = None).to_numpy()
        if rot_tir_flag:
            self.tir = np.rot90(self.tir, k = 2)
        
        self.Plot_RGB_TIR(self.rgb, self.tir, t_min=16, t_max=23)
        
    def Convergence(self, Point_rgb, Point_tir, crop_pix=20):
        # Совмещение RGB и TIR с помощью преобразования гомографии
        # crop_pix регулирует обрезку изображений с разных сторон
        
        H, _ = cv2.findHomography(Point_tir, Point_rgb) # Построение матрицы гомографии H по точкам совмещения
        self.tir_h = cv2.warpPerspective(self.tir, H, self.rgb.shape[:2]) # Преобразование TIR к RGB с помощью H
        
        # Обрезаем изображения так, чтобы область интереса была внутри ламп
        self.c1, self.c2 = Point_rgb[1][1] - crop_pix, Point_rgb[2][1] + 2 * crop_pix
        self.c3, self.c4 = Point_rgb[0][0] + crop_pix, Point_rgb[1][0] + crop_pix
        
        
        self.rgb_crop = self.rgb[self.c1 : self.c2, self.c3 : self.c4, :].copy()
        self.tir_crop = self.tir_h[self.c1 : self.c2, self.c3 : self.c4,]
        self.Plot_RGB_TIR(self.rgb_crop, self.tir_crop, t_min=16, t_max=23) # Иллюстрация для проверки точности совмещения
    
    def Plot_HSI(self):
        # Отображение HSI в виде псевдо RGB
        
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.axis('off')
        
        p_rgb = self.hsi[:, :, (70, 53, 19)] # Псевдо-rgb
        p_rgb = skimage.exposure.rescale_intensity(p_rgb, in_range=(0,50), out_range=(0,255)) # увеличение яркости
        plt.imshow(p_rgb)
        print(f'Размер HSI: {self.hsi.shape}')
    
    def Load_HSI(self, k_rot=3):
        # Загружаем HSI, отображаем псевдо RGB
        # HSI иногда перевернуто. Параметр k_rot отвечает за поворот HSI 
        # k_rot=1 поворачивает hsi 90 градусов, k_rot=2 на 180, k_rot=3 на 270, k_rot=0 на 0
        
        data, _ = util.load_ENVI_file(f'{self.path_hsi}/{self.folder}/{self.number}.hdr')
        self.hsi = np.rot90(data, k = k_rot)[self.c1 : self.c2, self.c3 : self.c4, :]
        
        self.Plot_HSI()
        
    def Save(self, path_out = 'TIR + HSI 90', flag_rot=False):
        # Сохранение данных как numpy массивы
        
        # Изображения (45 градусов) 6 и 25 дня должны быть перевернуты на 180 градусов перед сохранением (для 1 эксперимента)
        # Поворот необходим для того, чтобы справа были сухие части, а слева влажные
        if flag_rot:
            self.hsi = np.rot90(self.hsi, k = 2)
            self.tir_crop = np.rot90(self.tir_crop, k = 2)
            self.rgb_crop = np.rot90(self.rgb_crop, k = 2)
            
            
        path_lst = [path_out, str(self.folder)]
        Create_Folder(path_lst[0])
        path_out_full = '/'.join(path_lst)
        Create_Folder(path_out_full)
        np.save(f'{path_out_full}/HSI_{self.number}', self.hsi)
        np.save(f'{path_out_full}/TIR_{self.number}', self.tir_crop)
        np.save(f'{path_out_full}/RGB_{self.number}', self.rgb_crop)